In [2]:
import pandas as pd
import os
import numpy as np
import scipy as sp
import torch

In [3]:
from scipy.stats import uniform, norm
import seaborn as sns; sns.set()
import matplotlib.pyplot as plt

In [4]:
np.random.seed(0)
A = 1.2
b = -0.3
sigma = 0.2
N = 100

In [5]:
X = torch.from_numpy(uniform.rvs(size=N))
X.detach()
X = X.float()

In [6]:
e = torch.from_numpy(norm.rvs(loc=0, scale=sigma, size=N)).float()

In [7]:
y = A*X + b + e

In [8]:
y.detach()
y = y.float()

In [18]:
mu_A = 0.5
sigma_A = 0.3
qA = torch.distributions.normal.Normal(mu_A, sigma_A)

mu_b = -0.1
sigma_b = 0.1
qb = torch.distributions.normal.Normal(mu_b, sigma_b)

In [96]:
lst_lp1 = []
for j in range(0,100000):
    A = qA.sample()
    b = qb.sample()

    y_pred = A*X + b
    pData = torch.distributions.normal.Normal(y_pred, 0.2)

    lp = pData.log_prob(y).sum()
    lst_lp1.append(lp)
LP1 = torch.stack(lst_lp1)

In [97]:
pA = torch.distributions.normal.Normal(0.0, 1.0)
pb = torch.distributions.normal.Normal(0.0, 1.0)

In [98]:
F1 = torch.distributions.kl_divergence(qA, pA) + torch.distributions.kl_divergence(qb, pb) - LP1.mean()
F1

tensor(115.1674)

In [99]:
lst_lp2 = []
for j in range(0,100000):
    A = qA.sample()
    b = qb.sample()

    y_pred = A*X + b
    pData = torch.distributions.normal.Normal(y_pred, 0.2)

    lp = 0
    lp += qA.log_prob(A) + qb.log_prob(b)
    lp -= pA.log_prob(A) + pb.log_prob(b)
    lp -= pData.log_prob(y).sum()
    
    lst_lp2.append(lp)
LP2 = torch.stack(lst_lp2)

In [100]:
F2 = LP2.mean()
F2

tensor(115.3062)